In [37]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
from optimum.bettertransformer import BetterTransformer

In [38]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.1
Is CUDA enabled? True


In [39]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

Found cached dataset financial_phrasebank (C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


In [40]:
value_count_0=dataset['train']['label'].count(0)
value_count_1=dataset['train']['label'].count(1)
value_count_2=dataset['train']['label'].count(2)


print('number of negatives',value_count_0)
print('number of neutral',value_count_1)
print('number of positives',value_count_2)
#count_1=value_count['1'] # if it is str

number of negatives 604
number of neutral 2879
number of positives 1363


In [41]:

neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [42]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)


Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-9ab259d81c7af6d1.arrow
Loading cached split indices for dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-422398295eb4d128.arrow and C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-3283e3e9f0e377e4.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [43]:
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-501ca8eafc8dc863.arrow


In [44]:
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

model = model.to(0)


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=32, per_device_eval_batch_size=32)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [46]:
trainer.train()



                                             

                                       
  0%|          | 0/204 [00:18<?, ?it/s]


{'eval_loss': 0.2482908070087433, 'eval_accuracy': 0.9256198347107438, 'eval_runtime': 0.127, 'eval_samples_per_second': 952.743, 'eval_steps_per_second': 62.991, 'epoch': 1.0}




                                              

                                       
  0%|          | 0/204 [00:21<?, ?it/s]


{'eval_loss': 0.1252838671207428, 'eval_accuracy': 0.9752066115702479, 'eval_runtime': 0.121, 'eval_samples_per_second': 1000.005, 'eval_steps_per_second': 66.116, 'epoch': 2.0}




                                              

                                       
  0%|          | 0/204 [00:24<?, ?it/s]

100%|██████████| 204/204 [00:09<00:00, 22.65it/s]

{'eval_loss': 0.11935954540967941, 'eval_accuracy': 0.9752066115702479, 'eval_runtime': 0.124, 'eval_samples_per_second': 975.813, 'eval_steps_per_second': 64.517, 'epoch': 3.0}
{'train_runtime': 9.0045, 'train_samples_per_second': 362.152, 'train_steps_per_second': 22.655, 'train_loss': 0.2504114450192919, 'epoch': 3.0}


TrainOutput(global_step=204, training_loss=0.2504114450192919, metrics={'train_runtime': 9.0045, 'train_samples_per_second': 362.152, 'train_steps_per_second': 22.655, 'train_loss': 0.2504114450192919, 'epoch': 3.0})

In [47]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=128)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 1000.91it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cac

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [48]:
model = BetterTransformer.transform(model)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [49]:
trainer2.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

100%|██████████| 52/52 [00:00<00:00, 70.37it/s]


{'eval_loss': 0.6402676701545715,
 'eval_accuracy': 0.7944038929440389,
 'eval_runtime': 0.768,
 'eval_samples_per_second': 1070.311,
 'eval_steps_per_second': 67.708}